# Advanced Graph RAG: Powering Investment Intelligence with Knowledge Graphs

*By Saumil Srivastava, AI Engineering Consultant*

*<font color="#0F172A">Target Audience: Engineering Leaders, CTOs, Engineering Managers, Product Managers (AI Products)</font>*

---

## The Challenge: Moving Beyond Siloed Data in Investment Analysis

In the world of investment and finance, data is abundant: market prices, company fundamentals, news articles, economic indicators, fund prospectuses. However, this data often resides in disconnected silos. Answering complex, strategic questions can require analysts to manually piece together information from disparate sources, a time-consuming and often incomplete process.

**Consider questions like:**
* "Which S&P 500 companies in the 'Technology' sector, with a P/E ratio under 25 and positive recent news sentiment, are also significant holdings in our firm's top-performing ESG-focused ETFs?"
* "What is the interconnected risk exposure if a major supplier to several companies in our portfolio faces disruption, considering their industries and shared investors?"
* "Identify emerging competitors to our key holdings based on their technology focus, recent funding, and key personnel hires, even if they are not yet direct market comparables."

Answering these efficiently and accurately requires understanding not just individual data points, but the *relationships* between them. This is where traditional databases and even simple vector search on text documents fall short.

## The Solution: Knowledge Graphs for Connected Insights

A **Knowledge Graph (KG)** represents data as a network of entities (like companies, investors, financial instruments, news events) and the rich relationships that connect them. When combined with **Retrieval Augmented Generation (RAG)** powered by Large Language Models (LLMs), this "Graph RAG" approach allows us to:

1.  **Ask Complex, Multi-Hop Questions:** Traverse multiple layers of relationships to uncover non-obvious connections.
2.  **Integrate Diverse Data Types:** Seamlessly link structured financial data, unstructured news text, and inferred insights.
3.  **Enhance Context for LLMs:** Provide LLMs with precise, relevant, and interconnected information, leading to more accurate and insightful answers.
4.  **Drive Explainable AI:** The graph structure makes it easier to understand *why* an AI system arrived at a particular conclusion or recommendation.

This notebook provides a practical walkthrough of building an "Investment Intelligence KG." We'll demonstrate how to construct such a graph from public data, enrich it using LLMs, and then employ advanced Graph RAG techniques to answer sophisticated questions.

**For Engineering Leaders & CTOs:** This is about building next-generation data infrastructure for AI that offers a competitive edge in insight generation and operational efficiency.
**For Engineering Managers & Product Managers:** This is about equipping your teams with powerful methods to build more intelligent, context-aware applications and features that deliver tangible value.

**What We'll Cover:**
1.  **Data Foundation:** Using S&P 500 company data, `yfinance` for financial details & news, and a sample ETF list.
2.  **Building the Investment KG:** Defining entities (Companies, Stocks, ETFs, News, Sectors, Metrics) and their relationships.
3.  **LLM-Powered Graph Enrichment:** Inferring sentiment from news, generating company risk/opportunity summaries.
4.  **Advanced Graph RAG:**
    * **Text2Cypher:** LLMs generating Neo4j Cypher queries from natural language.
    * **Vector Search on Graph Data:** Finding semantically similar financial entities or concepts.
5.  **Strategic Advantage:** Understanding when Graph RAG is superior to simpler vector RAG for financial use cases.

Let's begin by setting up our environment and loading the foundational data.



In [6]:
# --- 0. Install necessary libraries ---
# Using versions known to be compatible for stability

%pip install langchain==0.1.17 neo4j==5.20.0 openai tiktoken==0.7.0 json-repair==0.39.1 yfinance pyvis requests-ratelimiter==0.4.2 langchain-openai datasets

In [3]:
# Downgrade google-genai to the last release that accepted httpx 0.27.x
%pip install --upgrade "google-genai<1.15.0"  # 1.14.0 as of today
%pip install --force-reinstall "httpx==0.27.2"


  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
  Attempting uninstall: httpx
    Found existing installation: httpx 0.27.2
    Uninstalling httpx-0.27.2:
      Successfully uninstalled httpx-0.27.2
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached certifi-2025.4.26-py3-none-any.whl.metadata (2.5 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
  Using cached typing_extensions-4.13.2-py3-none-any.whl.metadata (3.0 kB)
Using cached httpx-0.27.2-py3-none-any.whl (76 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached anyio-4.9.0-py3-none-any.whl (100 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached sni

In [ ]:


import os
import json
import pandas as pd
from typing import Optional, List, Dict, Any
from getpass import getpass
from io import StringIO # To read string data as CSV
import time
import yfinance as yf # For fetching stock data and news
import ast # For safely evaluating string-formatted lists

from google.colab import userdata

from openai import OpenAI
from neo4j import GraphDatabase, basic_auth

from requests_ratelimiter import LimiterSession, RequestRate, Limiter, Duration # Added for yfinance rate limiting
import ast # For safely evaluating string-formatted lists
from datasets import load_dataset, Dataset, DatasetDict
import logging
logger = logging.getLogger('my_logger')

logging.basicConfig()


print("--- Library Installation Complete ---")

# --- 1. Configure OpenAI API Key ---
OPENAI_API_KEY = ""
if os.getenv("OPENAI_API_KEY"):
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
else:
    try:
        OPENAI_API_KEY = userdata.get("OPENAI_API_KEY") # Using a specific key name
    except userdata.SecretNotFoundError:
        print("OpenAI API Key 'OPENAI_API_KEY_MAVEN' not found in Colab Secrets.")
        OPENAI_API_KEY = getpass("Please enter your OpenAI API Key: ")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

if not OPENAI_API_KEY.startswith("sk-"):
    print("⚠️ OpenAI API Key does not look valid. Please check.")
else:
    print("✅ OpenAI API key configured.")

# --- 2. Configure Neo4j Credentials ---
NEO4J_URI = ""
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = ""

try:
    NEO4J_URI = userdata.get('NEO4J_URI') # Using a specific key name
    NEO4J_PASSWORD = userdata.get('NEO4J_PASSWORD') # Using a specific key name
    if NEO4J_URI and NEO4J_PASSWORD:
         print("✅ Neo4j credentials loaded from Colab Secrets.")
    else:
        raise userdata.SecretNotFoundError
except userdata.SecretNotFoundError:
    print("Neo4j credentials ('NEO4J_URI_MAVEN', 'NEO4J_PASSWORD_MAVEN') not found or incomplete in Colab Secrets.")
    NEO4J_URI = input("Enter your Neo4j URI (e.g., neo4j+s://your-instance.databases.neo4j.io or bolt://localhost:7687): ")
    NEO4J_PASSWORD = getpass("Neo4j Password: ")

neo4j_driver = None
if NEO4J_URI and NEO4J_PASSWORD:
    try:
        neo4j_driver = GraphDatabase.driver(NEO4J_URI, auth=basic_auth(NEO4J_USERNAME, NEO4J_PASSWORD))
        neo4j_driver.verify_connectivity()
        print("✅ Connected to Neo4j database.")
    except Exception as e:
        print(f"🛑 Error connecting to Neo4j: {e}")
else:
    print("🛑 Neo4j URI or Password not provided. Please set them to continue.")


# --- 3. Initialize OpenAI Client ---
openai_client = OpenAI()

print("\n--- Setup Complete ---")
print(f"Neo4j Driver Initialized: {True if neo4j_driver else False}")
print(f"OpenAI Client Initialized: {True if openai_client else False}")

print("\n--- Setup Complete ---")
print(f"Neo4j Driver Initialized: {True if neo4j_driver else False}")
print(f"OpenAI Client Initialized: {True if openai_client else False}")

## Part 1: Data Foundation - S&P 500 Companies, `yfinance`, & Sample ETFs

Our Investment Intelligence KG will be built from three main sources for this demonstration:

1.  **S&P 500 Company List:** A foundational list of large-cap U.S. companies, their tickers, names, and sectors.
    * *Source Example for full dataset:* You can find S&P 500 component lists on Kaggle (e.g., search "S&P 500 companies list csv") or other financial data sites.
2.  **`yfinance` Library:** We'll use this popular Python library to fetch:
    * Detailed company information (like business summaries, market cap, P/E ratios) for a *subset* of S&P 500 companies.
    * Recent news headlines related to these companies.
3.  **Sample ETF Data:** A small, manually defined list of ETFs, their investment focus, and some sample holdings (using tickers from our S&P 500 list). In a real system, this would come from a dedicated financial data provider.

**Focus on Practicality:** We'll use a small, embedded sample of S&P 500 companies and ETFs directly in the notebook to ensure it runs smoothly for everyone. The `yfinance` calls will be limited to a few companies to manage execution time and API call politeness. This approach balances demonstrating powerful techniques with ease of use for a learning environment.

In [ ]:
!pip install --upgrade datasets


In [3]:

# """
# --------------------------------------------------------------------------------------
#  Loading Data from Hugging Face Hub
# --------------------------------------------------------------------------------------
logging.info("\n--- Attempting to Load Foundational Data from Hugging Face Hub ---")


YOUR_HF_USERNAME = "s4um1l" #
HF_COMPANIES_DATASET_NAME = f"{YOUR_HF_USERNAME}/investment-kg-companies"
HF_INVESTORS_DATASET_NAME = f"{YOUR_HF_USERNAME}/investment-kg-investors"
HF_INVESTMENTS_DATASET_NAME = f"{YOUR_HF_USERNAME}/investment-kg-investments"
HF_ETFS_DATASET_NAME = f"{YOUR_HF_USERNAME}/investment-kg-etfs"

LOAD_FROM_HF = True # Set to True once you have pushed your datasets to the Hub

if LOAD_FROM_HF and YOUR_HF_USERNAME != "your-hf-username":
    try:
        logging.info(f"Loading datasets from Hugging Face Hub user: {YOUR_HF_USERNAME}...")

        companies_hf = load_dataset(HF_COMPANIES_DATASET_NAME)
        detailed_companies_data = list(companies_hf["train"])

        investors_hf = load_dataset(HF_INVESTORS_DATASET_NAME)
        investor_df = investors_hf["train"].to_pandas()

        investments_hf = load_dataset(HF_INVESTMENTS_DATASET_NAME)
        investment_relationships_df = investments_hf["train"].to_pandas()

        etfs_hf = load_dataset(HF_ETFS_DATASET_NAME)
        etf_df = etfs_hf["train"].to_pandas()

        # Post-processing for loaded data (e.g., JSON parsing, datetime conversion)
        for company_data_item in detailed_companies_data:
            if 'news' in company_data_item and isinstance(company_data_item['news'], str):
                try:
                    company_data_item['news'] = json.loads(company_data_item['news'])
                    for news_item in company_data_item.get("news", []):
                        if news_item.get("publishTime") and isinstance(news_item.get("publishTime"), str):
                            try:
                                news_item["publishTime"] = pd.to_datetime(news_item["publishTime"])
                            except Exception as e:
                                logging.warning(f"Could not parse publishTime '{news_item['publishTime']}' from HF data: {e}")
                                news_item["publishTime"] = None
                except json.JSONDecodeError:
                    logging.warning(f"Could not parse news JSON for {company_data_item.get('ticker')}")
                    company_data_item['news'] = []

        logging.info("✅ Data successfully loaded from Hugging Face Hub.")
        # Set sp500_df_base from the loaded company data for consistency if needed by other cells
        if detailed_companies_data:
            sp500_df_base = pd.DataFrame([
                {"Symbol": c.get("ticker"), "Name": c.get("name"), "Sector": c.get("sector")}
                for c in detailed_companies_data
            ])

    except Exception as e:
        logging.error(f"🛑 Failed to load data from Hugging Face Hub: {e}")
        logging.warning("Falling back to using the embedded sample data defined in this cell.")
        LOAD_FROM_HF = False

# if not LOAD_FROM_HF:
#     logging.info("Using embedded sample data as LOAD_FROM_HF is False or loading failed.")
#     detailed_companies_data = detailed_companies_data_for_hf
#     investor_df = investor_df_for_hf
#     investment_relationships_df = investment_relationships_df_for_hf
#     etf_df = etf_df_for_hf
#     # Convert publishTime for embedded news (which were stored as strings for HF prep)
#     # Also, convert news JSON string back to list of dicts for detailed_companies_data
#     for company in detailed_companies_data:
#         if 'news' in company and isinstance(company['news'], str):
#             try:
#                 company['news'] = json.loads(company['news'])
#             except json.JSONDecodeError:
#                 logging.warning(f"Could not parse embedded news JSON for {company.get('ticker')}")
#                 company['news'] = []

#         for news_item in company.get("news", []):
#             if isinstance(news_item.get("publishTime"), str):
#                 try: news_item["publishTime"] = pd.to_datetime(news_item["publishTime"])
#                 except Exception as e:
#                     logging.warning(f"Could not parse publishTime '{news_item['publishTime']}' for {company['ticker']} (fallback): {e}")
#                     news_item["publishTime"] = None


logging.info("\n--- Foundational Data Ready for KG Population ---")

# --- Optional: Display sample of the prepared data ---
if detailed_companies_data:
    logging.info("\nSample of prepared detailed_companies_data (first company):")
    temp_print_data = detailed_companies_data[0].copy()
    if 'news' in temp_print_data and temp_print_data['news'] is not None:
        # Ensure news is a list of dicts before trying to process publishTime
        if isinstance(temp_print_data['news'], list):
            temp_print_data['news'] = [news_item.copy() for news_item in temp_print_data['news']]
            for news_item in temp_print_data['news']:
                if isinstance(news_item.get('publishTime'), pd.Timestamp):
                    news_item['publishTime'] = news_item['publishTime'].isoformat()
                elif news_item.get('publishTime') is None:
                     news_item['publishTime'] = 'N/A'
        elif isinstance(temp_print_data['news'], str): # If it's still a JSON string (e.g. fallback path didn't re-parse)
            try:
                parsed_news = json.loads(temp_print_data['news'])
                temp_print_data['news'] = [news_item.copy() for news_item in parsed_news]
                for news_item in temp_print_data['news']: # Process again after parsing
                    if isinstance(news_item.get('publishTime'), pd.Timestamp):
                        news_item['publishTime'] = news_item['publishTime'].isoformat()
                    elif news_item.get('publishTime') is None:
                        news_item['publishTime'] = 'N/A'
            except json.JSONDecodeError:
                 logging.warning(f"Could not parse news for printing for {temp_print_data.get('ticker')}")


    print(json.dumps(temp_print_data, indent=2))

if not investor_df.empty:
    logging.info("\nSample of prepared investor_df:")
    print(investor_df.head().to_string())

if not investment_relationships_df.empty:
    logging.info("\nSample of prepared investment_relationships_df:")
    print(investment_relationships_df.head().to_string())

if not etf_df.empty:
    logging.info("\nSample of prepared etf_df:")
    print(etf_df[['ticker', 'name', 'investment_focus', 'asset_class', 'expenseRatio', 'AUM', 'top_holdings_tickers']].head().to_string())


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/960 [00:00<?, ?B/s]

(…)-00000-of-00001-fbb02b735ef90dd7.parquet:   0%|          | 0.00/12.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/15 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/549 [00:00<?, ?B/s]

...

Generating train split:   0%|          | 0/14 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/716 [00:00<?, ?B/s]

(…)-00000-of-00001-9f4942236558f881.parquet:   0%|          | 0.00/5.50k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7 [00:00<?, ? examples/s]

{
  "ticker": "AAPL",
  "name": "Apple Inc.",
  "sector": "Information Technology",
  "industry": "Technology Hardware, Storage & Peripherals",
  "country": "USA",
  "summary": "Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. Known for innovation in consumer electronics and strong brand loyalty.",
  "marketCap": 2800000000000,
  "trailingPE": 28.5,
  "forwardPE": 26.0,
  "beta": 1.2,
  "website": "https://www.apple.com",
  "recommendationKey": "buy",
  "earningsDate": "2024-07-25",
  "news": [
    {
      "title": "Apple Vision Pro 2 Development Ramping Up",
      "publisher": "Tech Futures",
      "link": "https://example.com/news/apple-visionpro2",
      "publishTime": "2024-05-20T09:00:00+00:00"
    },
    {
      "title": "Apple Announces Quarterly Earnings Beat Expectations",
      "publisher": "MarketPulse",
      "link": "https://example.com/news/apple-earnings-q2",
      "publishTime": "2024-04-28T16:

In [6]:
import networkx as nx
from pyvis.network import Network
from IPython.display import HTML, display
import logging

# Ensure logging is configured
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Ensure detailed_companies_data and etf_df are available from Cell 4
if 'detailed_companies_data' not in locals() or 'etf_df' not in locals():
    logging.error("🛑 Foundational data (detailed_companies_data or etf_df) not loaded. Please run Cell 4 first.")
else:
    logging.info("\n--- Building In-Memory Graph with NetworkX ---")

    G = nx.DiGraph() # Directed graph

    # Define colors and sizes for node types for Pyvis
    node_colors = {
        "Company": "skyblue",
        "ETF": "lightgreen",
        "NewsArticle": "lightcoral",
        "Sector": "gold",
        "Industry": "khaki",
        "AssetClass": "lightpink"
    }
    node_sizes = {
        "Company": 25,
        "ETF": 20,
        "NewsArticle": 15,
        "Sector": 18,
        "Industry": 18,
        "AssetClass": 18
    }

    # To keep the visualization manageable for the demo, let's process only a subset
    # For example, the first 2-3 companies and related ETFs.
    # We'll use all companies from detailed_companies_data as it's already small.

    processed_nodes = set()

    # Add Company nodes and their related Sector, Industry, NewsArticle nodes
    for company_info in detailed_companies_data:
        if company_info.get("error"):
            logging.warning(f"Skipping company {company_info.get('ticker')} due to previous fetch error.")
            continue

        company_ticker = company_info["ticker"]
        company_name = company_info.get("name", company_ticker)

        if company_ticker not in processed_nodes:
            G.add_node(
                company_ticker,
                label=company_name[:30], # Pyvis label
                title=f"Ticker: {company_ticker}\nName: {company_name}\nSector: {company_info.get('sector', 'N/A')}\nIndustry: {company_info.get('industry', 'N/A')}\nSummary: {company_info.get('summary', 'N/A')[:100]}...", # Pyvis hover title
                entity_type="Company",
                color=node_colors["Company"],
                size=node_sizes["Company"],
                sector=company_info.get("sector"),
                industry=company_info.get("industry"),
                summary=company_info.get("summary")
            )
            processed_nodes.add(company_ticker)

        # Sector node and relationship
        sector_name = company_info.get("sector")
        if sector_name and sector_name != "N/A":
            if sector_name not in processed_nodes:
                G.add_node(sector_name, label=sector_name, title=f"Sector: {sector_name}", entity_type="Sector", color=node_colors["Sector"], size=node_sizes["Sector"])
                processed_nodes.add(sector_name)
            G.add_edge(company_ticker, sector_name, title="OPERATES_IN_SECTOR")

        # Industry node and relationship
        industry_name = company_info.get("industry")
        if industry_name and industry_name != "N/A":
            if industry_name not in processed_nodes:
                G.add_node(industry_name, label=industry_name, title=f"Industry: {industry_name}", entity_type="Industry", color=node_colors["Industry"], size=node_sizes["Industry"])
                processed_nodes.add(industry_name)
            G.add_edge(company_ticker, industry_name, title="OPERATES_IN_INDUSTRY")

        # NewsArticle nodes and relationships
        for news_item in company_info.get("news", []):
            news_title = news_item.get("title")
            news_link = news_item.get("link") # Use link as a more unique ID if titles can repeat

            if news_title and news_link: # Ensure there's a title and link
                news_node_id = news_link # Use link as node ID for uniqueness
                if news_node_id not in processed_nodes:
                    G.add_node(
                        news_node_id,
                        label=news_title[:40] + "...", # Pyvis label
                        title=f"Title: {news_title}\nPublisher: {news_item.get('publisher', 'N/A')}\nLink: {news_link}", # Pyvis hover title
                        entity_type="NewsArticle",
                        color=node_colors["NewsArticle"],
                        size=node_sizes["NewsArticle"],
                        publisher=news_item.get("publisher"),
                        publishTime=str(news_item.get("publishTime")) # Convert datetime to string for attributes
                    )
                    processed_nodes.add(news_node_id)
                G.add_edge(company_ticker, news_node_id, title="MENTIONED_IN_NEWS")

    # Add ETF nodes and their related AssetClass and Company (holdings) nodes
    if not etf_df.empty:
        for _, etf_row in etf_df.iterrows():
            if pd.notna(etf_row.get("error")): # Skip if ETF fetch had an error
                logging.warning(f"Skipping ETF {etf_row.get('ticker')} due to previous fetch error.")
                continue

            etf_ticker = etf_row["ticker"]
            etf_name = etf_row.get("name", etf_ticker)

            if etf_ticker not in processed_nodes:
                G.add_node(
                    etf_ticker,
                    label=etf_name[:30],
                    title=f"Ticker: {etf_ticker}\nName: {etf_name}\nFocus: {etf_row.get('investment_focus', 'N/A')[:100]}...",
                    entity_type="ETF",
                    color=node_colors["ETF"],
                    size=node_sizes["ETF"],
                    investment_focus=etf_row.get("investment_focus"),
                    asset_class_name=etf_row.get("asset_class") # Store asset class name for linking
                )
                processed_nodes.add(etf_ticker)

            # AssetClass node and relationship
            asset_class_name = etf_row.get("asset_class")
            if asset_class_name and asset_class_name != "N/A":
                if asset_class_name not in processed_nodes:
                    G.add_node(asset_class_name, label=asset_class_name, title=f"Asset Class: {asset_class_name}", entity_type="AssetClass", color=node_colors["AssetClass"], size=node_sizes["AssetClass"])
                    processed_nodes.add(asset_class_name)
                G.add_edge(etf_ticker, asset_class_name, title="FOCUSES_ON_ASSET_CLASS")

            # ETF_HOLDS_STOCK relationships
            for holding_ticker in etf_row.get("top_holdings_tickers", []):
                if G.has_node(holding_ticker) and G.has_node(etf_ticker): # Ensure both nodes exist
                    G.add_edge(etf_ticker, holding_ticker, title="HOLDS_STOCK")
                else:
                    logging.warning(f"Could not create HOLDS_STOCK edge for ETF {etf_ticker} to {holding_ticker} (one or both nodes missing in graph).")
    else:
        logging.warning("ETF DataFrame (etf_df) is empty. No ETF nodes or relationships will be added to NetworkX graph.")


    logging.info(f"✅ NetworkX graph built with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")

    # --- Visualize with Pyvis ---
    if G.number_of_nodes() > 0:
        logging.info("\n--- Generating Interactive Visualization with Pyvis (this might take a moment) ---")

        # Create a Pyvis network
        nt = Network(notebook=True, height="750px", width="100%", cdn_resources='in_line', directed=True)

        # Add nodes and edges from NetworkX graph to Pyvis network
        for node, attrs in G.nodes(data=True):
            nt.add_node(
                node,
                label=attrs.get('label', str(node)),
                title=attrs.get('title', str(node)),
                color=attrs.get('color', 'grey'),
                size=attrs.get('size', 15),
                shape='dot' # Default shape, can be customized further based on entity_type
            )

        for source, target, attrs in G.edges(data=True):
            nt.add_edge(source, target, title=attrs.get('title', 'RELATED_TO'))
        nt.toggle_physics(True)  # Enable force-directed algorithm

        # Configure physics and interaction for better layout and usability
        nt.set_options("""
        var options = {
          "physics": {
            "barnesHut": {
              "gravitationalConstant": -2000,
              "centralGravity": 0.2,
              "springLength": 120,
              "springConstant": 0.02,
              "damping": 0.09,
              "avoidOverlap": 0.1
            },
            "minVelocity": 0.75,
            "stabilization": {
                "enabled": true,
                "iterations": 200,
                "fit": true
            }
          },
          "interaction": {
            "hover": true,
            "tooltipDelay": 200,
            "hideEdgesOnDrag": true,
            "hideNodesOnDrag": false
          },
          "nodes": {
            "font": {
              "size": 12,
              "face": "Tahoma"
            }
          },
          "edges": {
            "arrows": {
              "to": { "enabled": true, "scaleFactor": 0.7 }
            },
            "color": {
              "inherit": "from"
            },
            "smooth": {
              "type": "continuous",
              "roundness": 0.2
            }
          }
        }
        """)
        nt.from_nx(G)
        # Save and show the graph
        try:
            file_name = "investment_kg_visualization.html"
            nt.save_graph(file_name)
            logging.info(f"✅ Pyvis graph saved to {file_name}")

            # Display in Colab (if in Colab environment)
            # For local Jupyter, nt.show(file_name) would open in a browser.
            # In Colab, we often display the HTML directly or provide a link.
            HTML(f'<p>Interactive graph saved as <a href="{file_name}" target="_blank">{file_name}</a>. Open it in a new tab for best experience.</p> <iframe src="{file_name}" width="100%" height="750px" style="border: 1px solid lightgrey;"></iframe>')

        except Exception as e:
            logging.error(f"🛑 Error generating or displaying Pyvis graph: {e}")
            logging.info("Pyvis visualization might not display correctly in all environments directly. Try opening the generated HTML file.")
    else:
        logging.warning("⚠️ NetworkX graph is empty. Skipping Pyvis visualization.")



In [8]:
HTML(file_name)

## Part 2: Building the Investment Knowledge Graph

With our foundational data loaded, we'll now construct the Neo4j Knowledge Graph. This involves creating nodes for each entity type and establishing meaningful relationships between them.

**Entities (Nodes) for our Investment KG:**
* `:Company` (Symbol (ticker), Name, Sector, Industry, Summary, MarketCap, P/E, Beta, Website)
* `:Stock` (Symbol/Ticker - often merged with Company or a distinct node linked to it)
* `:ETF` (Ticker, Name, InvestmentFocus, AssetClass)
* `:NewsArticle` (Title, Publisher, PublishTime, Link, InferredSentiment)
* `:Sector` (Name)
* `:Industry` (Name)
* `__Entity__` (Generic label for all nodes for unified indexing)

**Relationships:**
* `OPERATES_IN_SECTOR` (Company -> Sector)
* `OPERATES_IN_INDUSTRY` (Company -> Industry)
* `HAS_STOCK_DATA` (Company -> Stock, if Stock is a separate node, or properties on Company)
* `MENTIONED_IN_NEWS` (Company -> NewsArticle)
* `HAS_SENTIMENT` (NewsArticle -> SentimentNode, or as a property on NewsArticle)
* `ETF_HOLDS_STOCK` (ETF -> Company/Stock)
* `FOCUSES_ON_ASSET_CLASS` (ETF -> AssetClassNode)

**Strategic Importance of This Structure:**
This graph structure allows you to model the complex interplay between companies, their financial performance, market news, and investment vehicles like ETFs. It enables queries that can trace the impact of news on specific stocks, analyze ETF compositions based on company fundamentals, or identify companies within specific sectors that meet complex criteria. This interconnected view is often crucial for sophisticated investment analysis and is a core strength that graph databases bring to AI engineering in finance.

In [8]:
import logging
import pandas as pd # Ensure pandas is imported if not already
from typing import List, Dict, Any # Ensure these are imported

# Ensure logging is configured
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def create_investment_kg_constraints_updated(driver):
    """Creates unique constraints on node properties for data integrity and performance."""
    if not driver:
        logging.error("Neo4j driver not available. Skipping constraint creation.")
        return
    try:
        with driver.session(database="neo4j") as session: # Ensure correct database
            constraints = [
                "CREATE CONSTRAINT IF NOT EXISTS FOR (c:Company) REQUIRE c.ticker IS UNIQUE",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (s:Stock) REQUIRE s.ticker IS UNIQUE", # If Stock is a separate node
                "CREATE CONSTRAINT IF NOT EXISTS FOR (e:ETF) REQUIRE e.ticker IS UNIQUE",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (n:NewsArticle) REQUIRE n.link IS UNIQUE",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (sec:Sector) REQUIRE sec.name IS UNIQUE",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (ind:Industry) REQUIRE ind.name IS UNIQUE",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (ac:AssetClass) REQUIRE ac.name IS UNIQUE",
                "CREATE CONSTRAINT IF NOT EXISTS FOR (inv:Investor) REQUIRE inv.investorId IS UNIQUE", # New constraint for Investor
                "CREATE CONSTRAINT IF NOT EXISTS FOR (ent:__Entity__) REQUIRE ent.name IS UNIQUE"
            ]
            for constraint in constraints:
                session.run(constraint)
            logging.info("✅ Investment KG constraints (updated) created/ensured.")
    except Exception as e:
        logging.error(f"🛑 Error creating Investment KG constraints: {e}")

# Note: The clear_graph_data function was provided separately.
# Ensure it's run before populate_investment_kg_updated if needed.

def populate_investment_kg_updated(driver,
                                   companies_details: List[Dict],
                                   etf_details_df: pd.DataFrame,
                                   investor_info_df: pd.DataFrame, # New parameter
                                   investment_rels_df: pd.DataFrame # New parameter
                                   ):
    """Populates the Neo4j graph with expanded investment data."""
    if not driver:
        logging.error("Neo4j driver not available. Skipping KG population.")
        return

    logging.info("\n--- Populating Investment Knowledge Graph (Updated) ---")
    with driver.session(database="neo4j") as session:
        # 1. Process Companies and their details
        logging.info(f"Processing {len(companies_details)} companies...")
        for company_data in companies_details:
            if company_data.get("error"):
                logging.warning(f"  Skipping company {company_data.get('ticker', 'Unknown ticker')} due to previous fetch error.")
                continue

            ticker = company_data["ticker"]
            company_name = company_data.get("name", ticker)

            company_props = {
                "ticker": ticker,
                "name": company_name,
                "sector": company_data.get("sector", "N/A"),
                "industry": company_data.get("industry", "N/A"),
                "country": company_data.get("country", "N/A"), # New property
                "summary": company_data.get("summary", "N/A"),
                "marketCap": company_data.get("marketCap"), # Already handled if None
                "trailingPE": company_data.get("trailingPE"),
                "forwardPE": company_data.get("forwardPE"),
                "beta": company_data.get("beta"),
                "website": company_data.get("website", "N/A"),
                "recommendationKey": company_data.get("recommendationKey", "N/A"),
                "earningsDate": company_data.get("earningsDate", "N/A"),
                # investmentProfileSummary is added later by enrichment
            }

            # Remove None values from props to avoid issues with Neo4j properties
            company_props_cleaned = {k: v for k, v in company_props.items() if v is not None}


            company_cypher = """
            MERGE (c:Company {ticker: $props.ticker})
            SET c = $props // Overwrites/sets all properties from the props map
            WITH c
            CALL apoc.create.addLabels(c, ['__Entity__']) YIELD node
            SET node.name = $props.ticker
            RETURN distinct 'company_done'
            """
            session.run(company_cypher, props=company_props_cleaned)

            # Sector node and relationship
            if company_props_cleaned.get("sector") and company_props_cleaned["sector"] != "N/A":
                session.run("""
                    MERGE (s:Sector {name: $sector_name}) ON CREATE SET s.name = $sector_name
                    WITH s CALL apoc.create.addLabels(s, ['__Entity__']) YIELD node AS sector_node
                    MATCH (c:Company {ticker: $company_ticker})
                    MERGE (c)-[:OPERATES_IN_SECTOR]->(sector_node)
                """, sector_name=company_props_cleaned["sector"], company_ticker=ticker)

            # Industry node and relationship
            if company_props_cleaned.get("industry") and company_props_cleaned["industry"] != "N/A":
                session.run("""
                    MERGE (i:Industry {name: $industry_name}) ON CREATE SET i.name = $industry_name
                    WITH i CALL apoc.create.addLabels(i, ['__Entity__']) YIELD node AS industry_node
                    MATCH (c:Company {ticker: $company_ticker})
                    MERGE (c)-[:OPERATES_IN_INDUSTRY]->(industry_node)
                """, industry_name=company_props_cleaned["industry"], company_ticker=ticker)

            # News Articles
            for news_item in company_data.get("news", []):
                if news_item.get("link") and news_item.get("title"): # Ensure basic info exists
                    publish_time_str = None
                    if pd.notna(news_item.get("publishTime")):
                        if isinstance(news_item["publishTime"], pd.Timestamp):
                            publish_time_str = news_item["publishTime"].isoformat()
                        else: # Assume it's already an ISO string or compatible
                            publish_time_str = str(news_item["publishTime"])

                    news_props = {
                        "link": news_item["link"],
                        "title": news_item["title"],
                        "name": news_item["title"], # For __Entity__
                        "publisher": news_item.get("publisher", "N/A"),
                        "publishTime": publish_time_str
                    }
                    news_props_cleaned = {k: v for k, v in news_props.items() if v is not None}

                    session.run("""
                        MERGE (n:NewsArticle {link: $props.link})
                        SET n = $props
                        WITH n CALL apoc.create.addLabels(n, ['__Entity__']) YIELD node AS news_node
                        MATCH (c:Company {ticker: $company_ticker})
                        MERGE (c)-[:MENTIONED_IN_NEWS]->(news_node)
                    """, props=news_props_cleaned, company_ticker=ticker)
            logging.info(f"  Processed company: {company_name} ({ticker})")

        # 2. Process Investors
        if not investor_info_df.empty:
            logging.info(f"\nProcessing {len(investor_info_df)} investors...")
            for _, investor_row in investor_info_df.iterrows():
                investor_props = {
                    "investorId": investor_row["investorId"],
                    "name": investor_row["name"], # Also for __Entity__
                    "type": investor_row.get("type", "N/A"),
                    "focus_areas": investor_row.get("focus_areas", "N/A")
                }
                investor_props_cleaned = {k: v for k, v in investor_props.items() if v is not None}
                session.run("""
                    MERGE (inv:Investor {investorId: $props.investorId})
                    SET inv = $props
                    WITH inv CALL apoc.create.addLabels(inv, ['__Entity__']) YIELD node
                    // node.name is already set via props.name
                    RETURN distinct 'investor_done'
                """, props=investor_props_cleaned)
                logging.info(f"  Processed investor: {investor_props_cleaned['name']}")
        else:
            logging.warning("Investor DataFrame is empty. Skipping investor population.")

        # 3. Process Investment Relationships
        if not investment_rels_df.empty:
            logging.info(f"\nProcessing {len(investment_rels_df)} investment relationships...")
            for _, rel_row in investment_rels_df.iterrows():
                session.run("""
                    MATCH (inv:Investor {investorId: $investorId})
                    MATCH (c:Company {ticker: $companyTicker})
                    MERGE (inv)-[r:INVESTED_IN]->(c)
                    SET r.stage = $stage // Add stage as a relationship property
                    RETURN distinct 'investment_rel_done'
                """, investorId=rel_row["investorId"], companyTicker=rel_row["companyTicker"], stage=rel_row.get("stage", "N/A"))
            logging.info("  Investment relationships processed.")
        else:
            logging.warning("Investment relationships DataFrame is empty. Skipping.")

        # 4. Process ETFs and their holdings
        if not etf_details_df.empty:
            logging.info(f"\nProcessing {len(etf_details_df)} ETFs...")
            for _, etf_row in etf_details_df.iterrows():
                if pd.isna(etf_row.get("ticker")) or pd.isna(etf_row.get("name")):
                    logging.warning(f"  Skipping ETF due to missing ticker or name: {etf_row.get('ticker')}")
                    continue

                etf_ticker = etf_row["ticker"]
                etf_name = etf_row["name"]
                etf_props = {
                    "ticker": etf_ticker,
                    "name": etf_name, # Also for __Entity__
                    "investmentFocus": etf_row.get("investment_focus", "N/A"),
                    "assetClass": etf_row.get("asset_class", "N/A"),
                    "family": etf_row.get("family", "N/A"), # New property
                    "expenseRatio": etf_row.get("expenseRatio"), # Already float or None
                    "AUM": etf_row.get("AUM") # Already float or None
                }
                etf_props_cleaned = {k: v for k, v in etf_props.items() if v is not None}

                session.run("""
                    MERGE (e:ETF {ticker: $props.ticker})
                    SET e = $props
                    WITH e CALL apoc.create.addLabels(e, ['__Entity__']) YIELD node
                    // node.name is already set via props.name
                    RETURN distinct 'etf_done'
                """, props=etf_props_cleaned)

                # AssetClass node and relationship
                if etf_props_cleaned.get("assetClass") and etf_props_cleaned["assetClass"] != "N/A":
                     session.run("""
                        MERGE (ac:AssetClass {name: $ac_name}) ON CREATE SET ac.name = $ac_name
                        WITH ac CALL apoc.create.addLabels(ac, ['__Entity__']) YIELD node AS ac_node
                        MATCH (e:ETF {ticker: $etf_ticker})
                        MERGE (e)-[:FOCUSES_ON_ASSET_CLASS]->(ac_node)
                    """, ac_name=etf_props_cleaned["assetClass"], etf_ticker=etf_ticker)

                # Link ETF to Company (Stock) holdings using HOLDS_STOCK
                for holding_ticker in etf_row.get("top_holdings_tickers", []):
                    session.run("""
                        MATCH (e:ETF {ticker: $etf_ticker})
                        MATCH (c:Company {ticker: $company_ticker})
                        MERGE (e)-[r:HOLDS_STOCK]->(c)
                        SET r.source = 'sample_data'
                    """, etf_ticker=etf_ticker, company_ticker=holding_ticker)
                logging.info(f"  Processed ETF: {etf_name} ({etf_ticker})")
        else:
            logging.warning("ETF DataFrame is empty. Skipping ETF population.")

    logging.info("✅ Investment Knowledge Graph population complete (Updated).")

#--- Main Execution for KG Population (Example) ---
#This would be in a separate cell after Cell 4 (Data Loading) and the cell for clearing the DB.
if 'neo4j_driver' in locals() and neo4j_driver and \
   'detailed_companies_data' in locals() and \
   'etf_df' in locals() and \
   'investor_df' in locals() and \
   'investment_relationships_df' in locals():

    # 1. Clear existing data (IMPORTANT before re-populating)
    # clear_graph_data(neo4j_driver) # Call the function you have for this

    # 2. Create constraints
    create_investment_kg_constraints_updated(neo4j_driver)

    # 3. Populate graph
    populate_investment_kg_updated(
        neo4j_driver,
        detailed_companies_data,
        etf_df,
        investor_df,
        investment_relationships_df
    )
else:
    logging.warning("⚠️ Neo4j driver or necessary DataFrames not initialized. Cannot populate Investment KG.")
    logging.warning(f"  Driver: {'Present' if 'neo4j_driver' in locals() and neo4j_driver else 'Missing'}")
    logging.warning(f"  detailed_companies_data: {'Present' if 'detailed_companies_data' in locals() else 'Missing'}")
    logging.warning(f"  etf_df: {'Present' if 'etf_df' in locals() else 'Missing'}")
    logging.warning(f"  investor_df: {'Present' if 'investor_df' in locals() else 'Missing'}")
    logging.warning(f"  investment_relationships_df: {'Present' if 'investment_relationships_df' in locals() else 'Missing'}")



### Verifying the Investment KG Structure

Let's run some queries to ensure our Investment KG has been created as expected.

*(This is a good point, Saumil, to remind your audience that visualizing parts of the graph in the Neo4j Browser can be very insightful for understanding these connections.)*

In [9]:
def run_cypher_query_for_verification(driver, query, params=None):
    if not driver:
        print("Neo4j driver not available.")
        return []
    try:
        with driver.session(database="neo4j") as session:
            result = session.run(query, params)
            return [record.data() for record in result]
    except Exception as e:
        print(f"🛑 Cypher query error: {e}")
        return []

if neo4j_driver:
    print("\n--- Verifying Investment KG Structure ---")

    node_counts_query = """
    CALL db.labels() YIELD label
    CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) AS count', {}) YIELD value
    RETURN label, value.count AS count
    ORDER BY label
    """
    node_counts = run_cypher_query_for_verification(neo4j_driver, node_counts_query)
    print("\nNode Counts by Label:")
    if node_counts:
        for item in node_counts:
            print(f"  - {item['label']}: {item['count']}")
    else:
        print("  Could not retrieve node counts.")

    print("\nSample ETFs and their Holdings (Company Tickers):")
    sample_etf_holdings = run_cypher_query_for_verification(neo4j_driver, """
        MATCH (e:ETF)-[:HOLDS_STOCK]->(c:Company)
        RETURN e.name AS etf_name, e.ticker AS etf_ticker, collect(c.ticker) AS holdings_tickers
        LIMIT 3
    """)
    if sample_etf_holdings:
        for item in sample_etf_holdings:
            print(f"  - ETF: {item['etf_name']} ({item['etf_ticker']}), Holdings: {item['holdings_tickers']}")
    else:
        print("  No ETF holdings data found or error in query.")

    ticker_for_news_check = detailed_companies_data[0]['ticker'] if detailed_companies_data else None
    if ticker_for_news_check:
        print(f"\nNews for Company {ticker_for_news_check}:")
        company_news = run_cypher_query_for_verification(neo4j_driver, """
            MATCH (c:Company {ticker: $ticker})-[:MENTIONED_IN_NEWS]->(n:NewsArticle)
            RETURN n.title AS news_title, n.publisher AS publisher
            LIMIT 3
        """, params={"ticker": ticker_for_news_check})
        if company_news:
            for item in company_news:
                print(f"  - Title: {item['news_title']} (Publisher: {item['publisher']})")
        else:
            print(f"  No news found for {ticker_for_news_check} or error in query.")
    else:
        print("\nSkipping news check as no ticker was processed for detailed info.")
else:
    print("⚠️ Neo4j driver not initialized. Cannot run Investment KG verification queries.")


--- Verifying Investment KG Structure ---

Node Counts by Label:
  - AssetClass: 7
  - Company: 15
  - ETF: 7
  - Industry: 14
  - Investor: 5
  - NewsArticle: 16
  - Sector: 6
  - Stock: 0
  - __Entity__: 70

Sample ETFs and their Holdings (Company Tickers):
  - ETF: SPDR S&P 500 ETF Trust (SPY), Holdings: ['MSFT', 'NVDA', 'GOOGL', 'JPM', 'XOM', 'TSLA', 'AMZN', 'BRK-A', 'AAPL']
  - ETF: Invesco QQQ Trust (QQQ), Holdings: ['MSFT', 'NVDA', 'GOOGL', 'TSLA', 'AMZN', 'CRM', 'AAPL']
  - ETF: iShares ESG Aware MSCI USA ETF (ESGU), Holdings: ['MSFT', 'NVDA', 'GOOGL', 'TSLA', 'AAPL']

News for Company AAPL:
  - Title: Apple Vision Pro 2 Development Ramping Up (Publisher: Tech Futures)
  - Title: Apple Announces Quarterly Earnings Beat Expectations (Publisher: MarketPulse)


## Part 3: LLM-Powered Graph Enrichment - Extracting Deeper Insights with AI

Our Investment Knowledge Graph now contains foundational structured data. The next step, crucial for building truly intelligent systems, is to **enrich this graph using Large Language Models (LLMs)**. This involves leveraging LLMs to analyze existing data (both structured and unstructured) and infer new properties or relationships, making our KG more comprehensive and insightful.

**Why is LLM-driven enrichment valuable in an investment context?**

1.  **Sentiment Analysis at Scale:** LLMs can process news headlines or company reports to determine sentiment (positive, negative, neutral) towards a company or market trend. Storing this sentiment directly in the graph allows for powerful queries like, "Show me companies in the tech sector with recent positive news sentiment and strong P/E ratios."
2.  **Generating Actionable Summaries:** LLMs can synthesize information from multiple sources (e.g., a company's financial summary, recent news, sector trends) to create a concise `InvestmentProfileSummary` or `RiskOpportunityOutlook`. This provides analysts with quick, AI-generated perspectives.
3.  **Inferring Latent Attributes:** Based on a company's description, industry, and news, an LLM might infer attributes not explicitly stated, such as "PrimaryBusinessModel" (e.g., SaaS, B2C Marketplace) or "KeyTechnologicalFocus" (e.g., AI-driven drug discovery, Quantum computing).

**Our Enrichment Tasks for this Notebook:**

For our Investment KG, we will use an LLM to:

1.  **Infer Sentiment from News Headlines:** For each `NewsArticle` node linked to a `Company`, we'll use an LLM to classify the sentiment of the headline as `POSITIVE`, `NEGATIVE`, or `NEUTRAL`. This sentiment will be stored as a property on the `NewsArticle` node.
2.  **Generate an `InvestmentProfileSummary` for Companies:** For each `Company` node, we'll prompt an LLM to create a brief summary. The LLM will consider the company's existing `summary` (from yfinance), its `sector`, `industry`, and the aggregated sentiment from its recent news.

**The Engineering Perspective:**
Implementing such enrichment pipelines requires careful prompt engineering, managing LLM API calls (cost and rate limits), and designing efficient ways to update the graph. For engineering leaders, this highlights the need for MLOps practices even when primarily using pre-trained LLMs for inference tasks. The ROI comes from the significantly enhanced query capabilities and the depth of insight the enriched KG can provide.

Let's define our `InvestmentKGEnricher` class.

In [10]:
import time
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

class InvestmentKGEnricher:
    def __init__(self, openai_client: OpenAI, neo4j_driver: GraphDatabase.driver):
        self.client = openai_client
        self.driver = neo4j_driver
        self.model = "gpt-3.5-turbo-0125"

    def _get_llm_response(self, prompt_messages: List[Dict], retries=2, delay=5):
        if not self.client:
            logging.warning("OpenAI client not initialized. Skipping LLM call.")
            return None
        for attempt in range(retries):
            try:
                completion = self.client.chat.completions.create(
                    model=self.model,
                    messages=prompt_messages,
                    temperature=0.3,
                    response_format={"type": "json_object"}
                )
                content = completion.choices[0].message.content
                return json.loads(content)
            except json.JSONDecodeError as e_json:
                logging.error(f"LLM response was not valid JSON on attempt {attempt+1}: {content[:200]}... Error: {e_json}")
            except Exception as e:
                logging.error(f"LLM API call failed on attempt {attempt+1}: {e}")
            if attempt < retries - 1:
                logging.info(f"Retrying in {delay} seconds...")
                time.sleep(delay)
            else:
                logging.error("LLM call failed after multiple retries.")
        return None

    def infer_news_sentiment(self, news_article_link: str, news_title: str) -> Optional[str]:
        if not self.driver:
            logging.warning("Neo4j driver not available. Skipping news sentiment inference.")
            return None

        prompt_messages = [
            {"role": "system", "content": "You are a financial news sentiment analyzer. Classify the sentiment of the given news headline as POSITIVE, NEGATIVE, or NEUTRAL. Return ONLY a JSON object with a single key 'sentiment' and one of these three values."},
            {"role": "user", "content": f"News Headline: \"{news_title}\"\n\nJSON:"}
        ]

        llm_response = self._get_llm_response(prompt_messages)

        if llm_response and "sentiment" in llm_response:
            sentiment = llm_response["sentiment"].upper()
            if sentiment not in ["POSITIVE", "NEGATIVE", "NEUTRAL"]:
                logging.warning(f"LLM returned unexpected sentiment '{sentiment}' for title '{news_title}'. Defaulting to NEUTRAL.")
                sentiment = "NEUTRAL"

            try:
                with self.driver.session(database="neo4j") as session:
                    session.run("""
                        MATCH (n:NewsArticle {link: $link})
                        SET n.inferredSentiment = $sentiment
                    """, link=news_article_link, sentiment=sentiment)
                logging.info(f"  Updated sentiment for news '{news_title[:50]}...' to {sentiment}.")
                return sentiment
            except Exception as e:
                logging.error(f"  Failed to update sentiment in Neo4j for news '{news_title[:50]}...': {e}")
        else:
            logging.warning(f"  Could not determine sentiment for news '{news_title[:50]}...'. LLM response: {llm_response}")
        return None

    def generate_company_investment_profile(self, company_ticker: str) -> Optional[str]:
        if not self.driver:
            logging.warning("Neo4j driver not available. Skipping company profile generation.")
            return None

        query = """
        MATCH (c:Company {ticker: $ticker})
        OPTIONAL MATCH (c)-[:MENTIONED_IN_NEWS]->(n:NewsArticle)
        WHERE n.inferredSentiment IS NOT NULL
        RETURN c.name AS name, c.summary AS yfinanceSummary, c.sector AS sector, c.industry AS industry,
               collect(n.inferredSentiment) AS newsSentiments
        LIMIT 1
        """
        try:
            with self.driver.session(database="neo4j") as session:
                result = session.run(query, ticker=company_ticker).single()
        except Exception as e:
            logging.error(f"Failed to fetch company data from Neo4j for {company_ticker}: {e}")
            return None

        if not result:
            logging.warning(f"No data found in Neo4j for company ticker: {company_ticker}")
            return None

        company_name = result.get("name", company_ticker)
        yfinance_summary = result.get("yfinanceSummary", "No summary available.")
        sector = result.get("sector", "N/A")
        industry = result.get("industry", "N/A")
        news_sentiments = result.get("newsSentiments", [])

        positive_count = news_sentiments.count("POSITIVE")
        negative_count = news_sentiments.count("NEGATIVE")
        overall_sentiment_hint = "NEUTRAL"
        if positive_count > negative_count:
            overall_sentiment_hint = "LARGELY POSITIVE"
        elif negative_count > positive_count:
            overall_sentiment_hint = "LARGELY NEGATIVE"
        elif news_sentiments and positive_count == 0 and negative_count == 0:
             overall_sentiment_hint = "NEUTRAL"
        elif not news_sentiments:
            overall_sentiment_hint = "NOT AVAILABLE (NO RECENT NEWS WITH SENTIMENT)"

        prompt_messages = [
            {"role": "system", "content": "You are a financial analyst providing concise investment profile summaries. Based on the provided company information and recent news sentiment, generate a 1-2 sentence 'investmentProfileSummary'. Focus on key characteristics, market position, or potential outlook. Output ONLY a JSON object with a single key 'investmentProfileSummary'."},
            {"role": "user", "content": f"""
Company Name: {company_name}
Sector: {sector}
Industry: {industry}
Existing Summary: {yfinance_summary[:1000]} (Truncated for brevity)
Recent News Sentiment Trend: {overall_sentiment_hint}

JSON:
            """}
        ]

        llm_response = self._get_llm_response(prompt_messages)

        if llm_response and "investmentProfileSummary" in llm_response:
            profile_summary = llm_response["investmentProfileSummary"]
            try:
                with self.driver.session(database="neo4j") as session:
                    session.run("""
                        MATCH (c:Company {ticker: $ticker})
                        SET c.investmentProfileSummary = $summary
                    """, ticker=company_ticker, summary=profile_summary)
                logging.info(f"  Updated investment profile summary for {company_name}.")
                return profile_summary
            except Exception as e:
                logging.error(f"  Failed to update profile summary in Neo4j for {company_name}: {e}")
        else:
            logging.warning(f"  Could not generate investment profile summary for {company_name}. LLM response: {llm_response}")
        return None

    def enrich_investment_kg(self, company_tickers: List[str]):
        if not self.driver or not self.client:
            logging.error("Neo4j driver or OpenAI client not initialized. Aborting enrichment.")
            return

        logging.info("--- Starting Investment KG Enrichment Process ---")
        for i, ticker in enumerate(company_tickers):
            logging.info(f"\nProcessing enrichment for company ({i+1}/{len(company_tickers)}): {ticker}")

            news_to_process_query = """
            MATCH (c:Company {ticker: $ticker})-[:MENTIONED_IN_NEWS]->(n:NewsArticle)
            WHERE n.inferredSentiment IS NULL AND n.title IS NOT NULL AND n.link IS NOT NULL
            RETURN n.link AS link, n.title AS title
            LIMIT 5
            """
            try:
                with self.driver.session(database="neo4j") as session:
                    news_results = session.run(news_to_process_query, ticker=ticker)
                    news_items = [dict(record) for record in news_results]
            except Exception as e:
                logging.error(f"Failed to fetch news for {ticker} from Neo4j: {e}")
                continue

            if news_items:
                logging.info(f"  Found {len(news_items)} news articles to analyze for sentiment for {ticker}.")
                for news_item in news_items:
                    self.infer_news_sentiment(news_item["link"], news_item["title"])
                    time.sleep(0.5)
            else:
                logging.info(f"  No new news articles found to analyze for sentiment for {ticker}.")

            self.generate_company_investment_profile(ticker)

            if (i + 1) % 2 == 0 and len(company_tickers) > 2:
                logging.info("Pausing briefly to manage API rate limits...")
                time.sleep(3)

        logging.info("--- Investment KG Enrichment Process Complete ---")

if neo4j_driver and openai_client:
    enricher = InvestmentKGEnricher(openai_client=openai_client, neo4j_driver=neo4j_driver)

    if 'detailed_companies_data' in locals() and detailed_companies_data:
        tickers_to_enrich = [comp['ticker'] for comp in detailed_companies_data if not comp.get("error")]
        if tickers_to_enrich:
            enricher.enrich_investment_kg(tickers_to_enrich)
        else:
            logging.warning("No valid company tickers found from yfinance fetching to enrich.")
    else:
        logging.warning("Variable 'detailed_companies_data' not found or empty. Skipping enrichment.")

else:
    logging.error("Neo4j driver or OpenAI client not initialized. Cannot run enrichment.")

### Verifying Enriched Investment KG

Let's check our graph again to see the newly added `inferredSentiment` on `NewsArticle` nodes and the `investmentProfileSummary` on `Company` nodes.

In [11]:
def run_cypher_query_for_enrichment_verification(driver, query, params=None):
    if not driver:
        print("Neo4j driver not available.")
        return []
    try:
        with driver.session(database="neo4j") as session:
            result = session.run(query, params)
            return [record.data() for record in result]
    except Exception as e:
        print(f"🛑 Cypher query error: {e}")
        return []

if neo4j_driver:
    print("\n--- Verifying Enriched Investment KG Data ---")

    if 'detailed_companies_data' in locals() and detailed_companies_data:
        ticker_to_check = detailed_companies_data[0]['ticker']
        print(f"\nNews Sentiment for Company {ticker_to_check}:")
        news_sentiments = run_cypher_query_for_enrichment_verification(neo4j_driver, """
            MATCH (c:Company {ticker: $ticker})-[:MENTIONED_IN_NEWS]->(n:NewsArticle)
            WHERE n.inferredSentiment IS NOT NULL
            RETURN n.title AS news_title, n.inferredSentiment AS sentiment
            LIMIT 5
        """, params={"ticker": ticker_to_check})
        if news_sentiments:
            for item in news_sentiments:
                print(f"  - Title: {item['news_title'][:70]}... Sentiment: {item['sentiment']}")
        else:
            print(f"  No news with inferred sentiment found for {ticker_to_check}.")

    print("\nCompany Investment Profile Summaries (first 3 enriched):")
    enriched_company_profiles = run_cypher_query_for_enrichment_verification(neo4j_driver, """
        MATCH (c:Company)
        WHERE c.investmentProfileSummary IS NOT NULL
        RETURN c.name AS company, c.investmentProfileSummary AS profile_summary
        LIMIT 3
    """)
    if enriched_company_profiles:
        for item in enriched_company_profiles:
            print(f"  - Company: {item['company']}")
            print(f"    Profile Summary: {item['profile_summary']}")
    else:
        print("  No company investment profile summaries found.")
else:
    print("⚠️ Neo4j driver not initialized. Cannot run verification queries for enriched data.")


--- Verifying Enriched Investment KG Data ---

News Sentiment for Company AAPL:
  - Title: Apple Vision Pro 2 Development Ramping Up... Sentiment: POSITIVE
  - Title: Apple Announces Quarterly Earnings Beat Expectations... Sentiment: POSITIVE

Company Investment Profile Summaries (first 3 enriched):
  - Company: MSFT
    Profile Summary: Microsoft Corporation is a leader in cloud computing and enterprise software, with a strong focus on AI integration and significant partnerships with OpenAI. Recent news sentiment is largely positive.
  - Company: NVDA
    Profile Summary: NVIDIA Corporation is a leading force in AI hardware innovation, enabling data center acceleration with its advanced GPUs and software platforms. With largely positive recent news sentiment, the company's market position remains strong.
  - Company: GOOGL
    Profile Summary: Alphabet Inc., parent company of Google, is a leader in search, advertising, cloud computing, and AI research with a diversified portfolio inc

## Part 4: Vector Indexing - Enabling Semantic Search on the KG

Our Knowledge Graph (KG) now holds a wealth of structured and semi-structured information, including company details, financial metrics, news headlines, inferred sentiment, and LLM-generated summaries. To unlock its full potential, we need to be able to query it not just based on exact matches or explicit relationships, but also based on *semantic similarity* or conceptual meaning.

This is where **vector indexing** comes into play.

**What are Vector Embeddings and Vector Indexes?**

1.  **Vector Embeddings:** These are numerical representations (lists of numbers, or vectors) of text or other data. LLMs (or specialized embedding models) are used to convert textual information (like company summaries, news titles, or even user queries) into these dense vector embeddings. The key idea is that texts with similar meanings will have mathematically similar vectors.
2.  **Vector Index:** A specialized data structure built within Neo4j (or other databases) that stores these embeddings and allows for very fast similarity searches. Given a query vector, the index can quickly find the N most similar vectors (and thus the N most semantically similar nodes) in the graph.

**Why Use Vector Indexing on Our Investment KG?**

* **Semantic Search for Companies/ETFs:** Find companies or ETFs based on conceptual descriptions, even if the exact keywords aren't present. For example: "Find innovative tech companies focused on sustainable energy solutions" or "Show me ETFs that align with a conservative growth strategy."
* **Linking Unstructured Queries to Structured Data:** A user might ask a vague question. Vector search can identify the most relevant nodes (e.g., specific companies or industries) in the graph, which can then be used as starting points for more precise, structured Cypher queries.
* **Content-Based Recommendation:** Find companies similar to a given company based on their summaries, news sentiment, or strategic focus areas.

**Implementation in Neo4j:**
Neo4j allows you to create vector indexes on node properties. We'll create an index on our `__Entity__` nodes, leveraging properties like `name`, `summary`, `investmentProfileSummary`, `title`, and `investmentFocus`.

**The Engineering Value:**
Integrating vector search directly within the graph database offers significant advantages over maintaining separate vector and graph databases. It simplifies the architecture, reduces data synchronization issues, and allows for powerful hybrid queries that combine graph patterns with semantic similarity in a single operation. This leads to more efficient, scalable, and maintainable AI systems.

Let's create the vector index.

In [12]:
import logging
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Neo4jVector

if not os.getenv("OPENAI_API_KEY"):
    logging.warning("OPENAI_API_KEY environment variable not set. Langchain components might fail.")
    if 'openai_client' in locals() and openai_client and openai_client.api_key:
         os.environ["OPENAI_API_KEY"] = openai_client.api_key
         logging.info("Set OPENAI_API_KEY for Langchain from initialized openai_client.")

def create_neo4j_vector_index(driver, openai_api_key_for_lc):
    if not driver:
        logging.error("Neo4j driver not available. Cannot create vector index.")
        return None
    if not openai_api_key_for_lc:
        logging.error("OpenAI API Key not available. Cannot create embeddings for vector index.")
        return None

    embedding_node_property = 'embedding'
    index_name = 'investmentEntityIndex'

    logging.info("Updating nodes with a 'searchableDescription' property for consistent embedding...")
    try:
        with driver.session(database="neo4j") as session:
            session.run("""
                MATCH (c:Company) WHERE c.summary IS NOT NULL OR c.investmentProfileSummary IS NOT NULL
                SET c.searchableDescription = coalesce(c.name, "") + " | Industry: " + coalesce(c.industry, "") + ". Summary: " + coalesce(c.investmentProfileSummary, coalesce(c.summary, ""))
            """)
            session.run("""
                MATCH (e:ETF) WHERE e.investmentFocus IS NOT NULL
                SET e.searchableDescription = coalesce(e.name, "") + " | Focus: " + coalesce(e.investmentFocus, "")
            """)
            session.run("""
                MATCH (n:NewsArticle) WHERE n.title IS NOT NULL
                SET n.searchableDescription = coalesce(n.title, "")
            """)
            session.run("MATCH (s:Sector) WHERE s.name IS NOT NULL SET s.searchableDescription = s.name")
            session.run("MATCH (i:Industry) WHERE i.name IS NOT NULL SET i.searchableDescription = i.name")
        logging.info("✅ 'searchableDescription' property updated/created for relevant nodes.")
    except Exception as e:
        logging.error(f"🛑 Error creating 'searchableDescription': {e}")
        logging.warning("Proceeding with vector index creation, but embeddings might be suboptimal.")

    logging.info(f"Attempting to create/retrieve Neo4jVector index: '{index_name}'")
    try:
        if not os.getenv("OPENAI_API_KEY"):
             os.environ["OPENAI_API_KEY"] = openai_api_key_for_lc

        embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

        vector_store = Neo4jVector.from_existing_graph(
            embedding=embeddings,
            url=NEO4J_URI,
            username=NEO4J_USERNAME,
            password=NEO4J_PASSWORD,
            index_name=index_name,
            node_label="__Entity__",
            text_node_properties=["searchableDescription"],
            embedding_node_property=embedding_node_property,
            database="neo4j"
        )
        logging.info(f"✅ Vector index '{index_name}' created/retrieved successfully.")
        return vector_store
    except Exception as e:
        logging.error(f"🛑 Failed to create/retrieve Neo4jVector index: {e}")
        return None

vector_retriever_store = None
if neo4j_driver and OPENAI_API_KEY:
    logging.info("Creating/retrieving vector index. This may take some time if populating for the first time...")
    vector_retriever_store = create_neo4j_vector_index(neo4j_driver, OPENAI_API_KEY)
else:
    logging.warning("Neo4j driver or OpenAI API Key not available. Skipping vector index creation.")

In [13]:
import logging

def test_similarity_search(vector_store_instance, query: str, k: int = 3):
    if not vector_store_instance:
        logging.error("Vector store instance not available. Cannot perform similarity search.")
        return []

    logging.info(f"\nPerforming similarity search for query: '{query}' (top {k} results)")
    try:
        results = vector_store_instance.similarity_search_with_score(query, k=k)
        if results:
            logging.info("✅ Similarity search successful. Results:")
            for i, (doc, score) in enumerate(results):
                logging.info(f"  Result {i+1} (Score: {score:.4f}):")
                logging.info(f"    Node ID: {doc.metadata.get('element_id', 'N/A')}")
                logging.info(f"    Labels: {doc.metadata.get('_labels', 'N/A')}")
                logging.info(f"    Content (searchableDescription): {doc.page_content[:200]}...")
                if 'name' in doc.metadata:
                     logging.info(f"    Name: {doc.metadata['name']}")
                if 'ticker' in doc.metadata:
                     logging.info(f"    Ticker: {doc.metadata['ticker']}")
        else:
            logging.warning("Similarity search returned no results.")
        return results
    except Exception as e:
        logging.error(f"🛑 Error during similarity search: {e}")
        return []

if 'vector_retriever_store' in locals() and vector_retriever_store:
    test_query_1 = "Technology companies innovating in artificial intelligence"
    search_results_1 = test_similarity_search(vector_retriever_store, test_query_1)

    test_query_2 = "ETFs focusing on sustainable and ESG investments"
    search_results_2 = test_similarity_search(vector_retriever_store, test_query_2)

    if 'detailed_companies_data' in locals() and detailed_companies_data and detailed_companies_data[0].get("news"):
        first_company_news_title_snippet = detailed_companies_data[0]["news"][0]["title"][:30] if detailed_companies_data[0]["news"] else "market performance"
        test_query_3 = f"Recent news about {first_company_news_title_snippet}"
        search_results_3 = test_similarity_search(vector_retriever_store, test_query_3)
    else:
        logging.info("Skipping news-related vector search test as no news data was processed.")
else:
    logging.warning("Vector store not initialized. Skipping similarity search tests.")

## Part 5: Advanced Retrieval Strategies - Querying the Investment KG with Intelligence

Our Investment Knowledge Graph (KG) is now populated with structured data, LLM-enriched insights (like company summaries and news sentiment), and a vector index for semantic search. The next crucial step is to implement sophisticated retrieval strategies that can effectively harness this rich, interconnected data to answer complex business questions.

We'll focus on two powerful approaches:

1.  **Text2Cypher Retriever:** This technique uses a Large Language Model (LLM) to translate a user's natural language question into a precise Cypher query (Neo4j's graph query language). This allows users to interact with the graph without needing to know Cypher, enabling highly specific and complex data retrieval based on graph patterns and relationships.
    * **Business Value:** Empowers business analysts or decision-makers to ask ad-hoc, detailed questions about company relationships, investment patterns, market structures, etc., that would be difficult or impossible with traditional SQL or simple keyword searches.

2.  **Vector Similarity Retriever (Graph-Aware):** This retriever uses the vector index to find nodes (Companies, ETFs, News) whose textual descriptions are semantically similar to the user's query.
    * **Business Value:** Useful for exploratory queries, finding entities based on conceptual descriptions rather than exact keywords, or for content-based recommendations.

**The Engineering Perspective:**
Implementing these retrievers involves careful prompt engineering for Text2Cypher, efficient interaction with the Neo4j database, and structuring the retrieved information so it can be effectively used by an LLM to generate a final answer. Performance considerations include optimizing Cypher queries generated by the LLM and managing the cost of LLM calls.

Let's start by building the `Text2CypherRetriever`.

In [15]:
import logging
import json
from neo4j import GraphDatabase
from openai import OpenAI
from typing import List, Dict, Any, Optional
import time

class Text2CypherInvestmentRetriever:
    """
    Converts natural language questions about the Investment KG into Cypher queries using LLMs,
    executes them, and can synthesize answers.

    Mermaid diagram for flow:
    ```mermaid
    graph TD
        A[User Question (Natural Language)] --> B{Text2CypherInvestmentRetriever Instance};
        B -- 1. Calls `generate_cypher_query` --> C[generate_cypher_query Method];
        C -- a. Accesses/Formats --> D[Graph Schema (from Neo4j)];
        C -- b. Constructs Prompt with Question & Schema --> E[LLM (Cypher Gen Model)];
        E -- Returns --> F[Generated Cypher Query String];
        F --> B;
        B -- 2. Calls `execute_cypher` with Cypher --> G[execute_cypher Method];
        G -- Executes Query --> H[(Neo4j Database)];
        H -- Returns Raw Results --> I[Query Results (Structured Data)];
        I --> B;
        B -- 3. Calls `synthesize_answer` with Question & Results --> J[synthesize_answer Method];
        J -- Constructs Prompt with Question & Results --> K[LLM (Answer Gen Model)];
        K -- Returns --> L[Synthesized Natural Language Answer];
        L --> M[Output to User];
    ```
    """
    def __init__(self, neo4j_driver: GraphDatabase.driver, openai_client: OpenAI, graph_schema: Optional[str] = None):
        self.driver = neo4j_driver
        self.client = openai_client
        self._schema = graph_schema
        self.cypher_gen_model = "gpt-4o-mini"
        self.answer_gen_model = "gpt-4.1-nano"

    @property
    def schema(self) -> str:
        """Lazily fetches and formats the Neo4j schema if not provided."""
        if self._schema is None:
            if not self.driver:
                logging.error("Neo4j driver not available to fetch schema.")
                return "Schema not available."
            try:
                logging.info("Fetching graph schema from Neo4j...")
                with self.driver.session(database="neo4j") as session:
                    schema_query = "CALL db.schema.visualization() YIELD nodes, relationships RETURN nodes, relationships"
                    result = session.run(schema_query).single()
                    if not result: return "Could not retrieve schema."

                    nodes_schema = []
                    valid_nodes = [node for node in result.get("nodes", []) if node is not None and node.get('name') is not None]
                    sorted_nodes = sorted(valid_nodes, key=lambda x: x.get('name', ""))
                    for node in sorted_nodes:
                        label = node.get('name', 'UnknownLabel')
                        properties = sorted([
                            prop['property'] for prop in node.get('properties', [])
                            if prop['property'] not in ['embedding', 'searchableDescription']
                        ])
                        prop_str = f", Properties: {properties}" if properties else ""
                        id_prop_note = ""
                        if not properties:
                           if label == "Company" or label == "ETF": id_prop_note = f" (Primarily identified by 'ticker')"
                           elif label == "NewsArticle": id_prop_note = f" (Primarily identified by 'link')"
                           elif label == "Investor": id_prop_note = f" (Primarily identified by 'investorId')"
                           else: id_prop_note = f" (Primarily identified by 'name')"
                        nodes_schema.append(f"Node Label: `{label}`{prop_str}{id_prop_note}")

                    rels_schema = []
                    valid_relationships = [rel for rel in result.get("relationships", []) if rel is not None and rel.get('type') is not None]
                    sorted_relationships = sorted(valid_relationships, key=lambda x: x.get('type', ""))
                    for rel in sorted_relationships:
                        rel_type = rel.get('type', 'UnknownRelationshipType')
                        start_node_label = rel.get('startNodeLabel', 'ANY')
                        end_node_label = rel.get('endNodeLabel', 'ANY')
                        rel_props = sorted([p_info['property'] for p_info in rel.get('properties', []) if not p_info['property'].startswith('_')])
                        prop_string = f", Properties: {rel_props}" if rel_props else ""
                        rels_schema.append(f"Relationship Type: `:{rel_type}` (Connects `{start_node_label}` to `{end_node_label}`{prop_string})")

                    self._schema = "Graph Schema Overview:\nNodes:\n" + "\n".join(nodes_schema) + "\n\nRelationships:\n" + "\n".join(list(set(rels_schema)))
                    logging.info("✅ Graph schema fetched and formatted for Text2Cypher.")
            except Exception as e:
                logging.error(f"Failed to fetch schema from Neo4j: {e}")
                self._schema = "Error fetching schema. Queries may be suboptimal."
        return self._schema

    def _get_llm_response_for_cypher(self, prompt_messages: List[Dict], retries=2, delay=5) -> Optional[str]:
        if not self.client:
            logging.warning("OpenAI client not initialized. Skipping Cypher generation.")
            return None
        for attempt in range(retries):
            try:
                completion = self.client.chat.completions.create(
                    model=self.cypher_gen_model,
                    messages=prompt_messages,
                    temperature=0.0,
                )
                cypher_query = completion.choices[0].message.content.strip()
                if cypher_query.startswith("```cypher"):
                    cypher_query = cypher_query[len("```cypher"):].strip()
                if cypher_query.startswith("```"):
                    cypher_query = cypher_query[3:].strip()
                if cypher_query.endswith("```"):
                    cypher_query = cypher_query[:-3].strip()
                return cypher_query
            except Exception as e:
                logging.error(f"LLM Cypher generation call failed on attempt {attempt+1}: {e}")
            if attempt < retries - 1:
                logging.info(f"Retrying Cypher generation in {delay} seconds...")
                time.sleep(delay)
        logging.error("LLM Cypher generation failed after multiple retries.")
        return None

    def generate_cypher_query(self, question: str) -> Optional[str]:
        system_prompt = f"""You are an expert Neo4j Cypher query writer specializing in financial and investment data.
Your task is to translate natural language questions into executable Cypher queries for a graph with the following schema details:
{self.schema}

Important Guidelines & Entity Identification:
- Company nodes (`Company`) are primarily identified by their 'ticker' property (e.g., {{ticker: 'AAPL'}}). Other properties include 'name', 'sector', 'industry', 'country', 'summary', 'investmentProfileSummary', 'marketCap', 'trailingPE', 'forwardPE', 'beta', 'website', 'recommendationKey', 'earningsDate'.
- ETF nodes (`ETF`) are identified by their 'ticker' property (e.g., {{ticker: 'SPY'}}). Other properties include 'name', 'investmentFocus', 'assetClass', 'family', 'expenseRatio', 'AUM', 'description_from_funds_data'.
- Investor nodes (`Investor`) are identified by their 'investorId' property. Other properties include 'name', 'type', 'focus_areas'.
- NewsArticle nodes (`NewsArticle`) are identified by their 'link' property. Other properties include 'title', 'publisher', 'publishTime', 'inferredSentiment' (POSITIVE, NEGATIVE, NEUTRAL).
- Sector nodes (`Sector`), Industry nodes (`Industry`), and AssetClass nodes (`AssetClass`) are identified by their 'name' property.

Relationships to use:
- `OPERATES_IN_SECTOR` (Company -> Sector)
- `OPERATES_IN_INDUSTRY` (Company -> Industry)
- `MENTIONED_IN_NEWS` (Company -> NewsArticle)
- `HOLDS_STOCK` (ETF -> Company)
- `INVESTED_IN` (Investor -> Company), this relationship has a 'stage' property (e.g., {{stage: 'Series A'}}).
- `FOCUSES_ON_ASSET_CLASS` (ETF -> AssetClass)

Query Construction Rules:
- Avoid using 'embedding' or 'searchableDescription' properties in MATCH clauses or WHERE conditions.
- When returning node properties, use explicit aliases without dots (e.g., `RETURN i.name AS investorName, i.type AS investorType`). This helps the answer synthesis step.
- For string comparisons (names, sectors, etc.), ALWAYS use case-insensitive matching: `toLower(node.property) CONTAINS toLower('search_term')` or `toLower(node.property) = toLower('exact_term')`.
- For "latest news", use `ORDER BY n.publishTime DESC`. Return a few (e.g., 3) items if "latest" is not strictly singular.
- Prioritize `ticker` for `Company`/`ETF` and `investorId` for `Investor` in MATCH patterns.
- Output ONLY the Cypher query.
"""

        prompt_messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Natural language question: \"{question}\"\n\nCypher Query:"}
        ]

        cypher_query = self._get_llm_response_for_cypher(prompt_messages)
        if cypher_query:
            logging.info(f"Generated Cypher for '{question}':\n{cypher_query}")
        return cypher_query

    def execute_cypher(self, cypher_query: str) -> List[Dict[str, Any]]:
        if not self.driver or not cypher_query:
            logging.error("Neo4j driver or Cypher query not available.")
            return []
        try:
            with self.driver.session(database="neo4j") as session:
                result = session.run(cypher_query)
                records = [record.data() for record in result]
                logging.info(f"Cypher query executed successfully. Fetched {len(records)} records.")
                return records
        except Exception as e:
            logging.error(f"Error executing Cypher query \"{cypher_query[:100]}...\": {e}")
            return [{"error": str(e)}]

    def synthesize_answer(self, question: str, query_results: List[Dict[str, Any]]) -> str:
        if not self.client:
            logging.warning("OpenAI client not initialized. Skipping answer synthesis.")
            return "Could not synthesize answer: OpenAI client not available."

        logging.info(f"Data for synthesis for question '{question}': {json.dumps(query_results, indent=2, default=str)}")

        if query_results and isinstance(query_results[0], dict) and "error" in query_results[0]:
            error_message = query_results[0]["error"]
            logging.warning(f"Error in query execution results for '{question}'. Error: {error_message}")
            return f"I couldn't find specific information for your query due to an error during data retrieval: {error_message}. Please try rephrasing or check the system logs."

        if not query_results:
             logging.warning(f"No results found to synthesize answer for '{question}'. Query returned an empty set.")
             return "Based on the information in the knowledge graph, I could not find any specific data matching your query."

        results_for_prompt = query_results[:10]
        results_str = json.dumps(results_for_prompt, indent=2, default=str)

        synthesis_system_prompt = """You are a financial analyst assistant. Your task is to answer the user's question based *only* on the structured data provided.

Instructions:
1.  Examine the "Original Question" and the "Retrieved Data" (in JSON format).
2.  If the "Retrieved Data" is an empty list `[]`, respond with: "Based on the information in the knowledge graph, I could not find any specific data matching your query."
3.  If the "Retrieved Data" is not empty:
    a.  If the question asks "Which [items]..." (e.g., "Which investors...", "Which companies..."), and the data is a list of objects, list the relevant information from these objects. For example, if data is `[{"investorName": "Alpha Fund", "investorType": "VC"}, {"investorName": "Beta Capital", "investorType": "PE"}]` and the question was "Which investors...", your answer should be something like: "The investors who funded both are Alpha Fund (Type: VC) and Beta Capital (Type: PE)." or "The following investors were found: Alpha Fund (VC), Beta Capital (PE)." Ensure you clearly state what the listed items represent in relation to the question.
    b.  If the question asks for a count (e.g., "How many companies...") and the data provides a count (e.g., `[{"numberOfCompanies": 3}]`), state the count: "There are 3 such companies."
    c.  For other types of questions, synthesize a concise answer using the properties present in the data.
4.  If the data is present but doesn't seem to directly or fully answer the question, state what you found and mention that it might not be a complete answer to the specific question.
5.  Do NOT make up information or infer beyond what is explicitly in the "Retrieved Data".
6.  Be professional and direct. Use the property names in the JSON (e.g., `investorName`, `investorType`, `sectorName`) to understand the data.
"""
        prompt_messages = [
            {"role": "system", "content": synthesis_system_prompt},
            {"role": "user", "content": f"""
Original Question: {question}

Retrieved Data (JSON format):
{results_str}

Answer:
"""}
        ]

        try:
            completion = self.client.chat.completions.create(
                model=self.answer_gen_model,
                messages=prompt_messages,
                temperature=0.05
            )
            answer = completion.choices[0].message.content.strip()
            logging.info(f"Synthesized answer for '{question}': {answer}")
            return answer
        except Exception as e:
            logging.error(f"LLM answer synthesis call failed: {e}")
            return "There was an issue generating the final answer."

    def query_kg(self, question: str) -> Dict[str, Any]:
        logging.info(f"\n--- Text2Cypher Processing Question: \"{question}\" ---")
        current_schema = self.schema
        if "Schema not available" in current_schema or "Error fetching schema" in current_schema:
            logging.error("Cannot generate Cypher query due to schema issues.")
            return {
                "question": question, "cypher_query": None, "results_count":0, "raw_results": [],
                "answer": "System error: Could not retrieve graph schema to process your query.", "retrieval_method": "Text2Cypher"
            }

        generated_cypher = self.generate_cypher_query(question)

        if not generated_cypher:
            return {
                "question": question, "cypher_query": None, "results_count": 0,
                "raw_results": [], "answer": "Failed to generate a Cypher query for your question.",
                "retrieval_method": "Text2Cypher"
            }

        results = self.execute_cypher(generated_cypher)
        answer = self.synthesize_answer(question, results)

        return {
            "question": question, "cypher_query": generated_cypher,
            "results_count": len(results) if not (results and isinstance(results[0], dict) and "error" in results[0]) else 0,
            "raw_results": results, "answer": answer, "retrieval_method": "Text2Cypher"
        }

# --- New Standalone Function to Explain Cypher ---
def explain_cypher_in_natural_language(
    cypher_query_string: str,
    graph_schema_string: str,
    openai_client: OpenAI,
    model: str = "gpt-3.5-turbo-0125" # Or gpt-4o-mini
) -> Optional[str]:
    """
    Uses an LLM to explain a Cypher query in natural language, given the graph schema.
    """
    if not openai_client:
        logging.error("OpenAI client not provided. Cannot explain Cypher.")
        return None
    if not cypher_query_string:
        logging.warning("Empty Cypher query string provided.")
        return "No Cypher query to explain."
    if "Schema not available" in graph_schema_string or "Error fetching schema" in graph_schema_string:
        logging.warning("Graph schema is not available or contains errors. Explanation quality may be affected.")

    system_prompt = """You are an expert at explaining Neo4j Cypher queries in simple, step-by-step natural language.
Given the following graph schema and Cypher query, provide a clear explanation of what the query is doing.
Describe:
1. Which types of nodes (entities) the query starts by looking for.
2. How it traverses relationships to find other related nodes.
3. What conditions or filters are applied to these nodes or relationships.
4. What information the query ultimately returns.
Make the explanation easy for someone not deeply familiar with Cypher to understand.
"""

    user_prompt = f"""
Graph Schema Overview:
---
{graph_schema_string}
---

Cypher Query to Explain:
---
{cypher_query_string}
---

Step-by-step Natural Language Explanation:
"""
    prompt_messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    try:
        completion = openai_client.chat.completions.create(
            model=model,
            messages=prompt_messages,
            temperature=0.2
        )
        explanation = completion.choices[0].message.content.strip()
        logging.info(f"Successfully explained Cypher query:\n{explanation}")
        return explanation
    except Exception as e:
        logging.error(f"LLM call for Cypher explanation failed: {e}")
        return "Could not generate an explanation for the Cypher query due to an error."

# --- Instantiate and Test Text2CypherRetriever ---
if 'neo4j_driver' in locals() and neo4j_driver and \
   'openai_client' in locals() and openai_client:

    text2cypher_retriever = Text2CypherInvestmentRetriever(
        neo4j_driver=neo4j_driver,
        openai_client=openai_client
    )
    logging.info(f"Schema for Text2Cypher (first 200 chars): \n{text2cypher_retriever.schema[:200]}...")

    # Test Query 1 (AAPL News & Sector)
    aapl_processed = any(comp.get('ticker') == 'AAPL' and not comp.get('error') for comp in detailed_companies_data) if 'detailed_companies_data' in locals() else False
    if aapl_processed:
        test_question_t2c_1 = "What is the latest news sentiment for Apple (AAPL) and its current sector?"
        response_t2c_1 = text2cypher_retriever.query_kg(test_question_t2c_1)
        print("\n--- Test Query 1 (Text2Cypher) ---")
        print(f"Question: {response_t2c_1['question']}")
        print(f"Generated Cypher:\n{response_t2c_1['cypher_query']}")
        print(f"Answer: {response_t2c_1['answer']}")
        if response_t2c_1['cypher_query']: # Explain if Cypher was generated
            print("\n--- Cypher Explanation for Query 1 ---")
            explanation_q1 = explain_cypher_in_natural_language(response_t2c_1['cypher_query'], text2cypher_retriever.schema, openai_client)
            print(explanation_q1)
    else:
        print("\nSkipping Test Query 1 for Text2Cypher (AAPL data not processed/available).")

    # Test Query 2 (MSFT, ESG ETFs, Summary)
    msft_processed = any(comp.get('ticker') == 'MSFT' and not comp.get('error') for comp in detailed_companies_data) if 'detailed_companies_data' in locals() else False
    esgu_exists = 'etf_df' in locals() and 'ticker' in etf_df.columns and not etf_df[etf_df['ticker'] == 'ESGU'].empty
    if msft_processed and esgu_exists:
        test_question_t2c_2 = "List ETFs that hold Microsoft (MSFT) and focus on ESG. Also, what is Microsoft's investment profile summary?"
        response_t2c_2 = text2cypher_retriever.query_kg(test_question_t2c_2)
        print("\n--- Test Query 2 (Text2Cypher) ---")
        print(f"Question: {response_t2c_2['question']}")
        print(f"Generated Cypher:\n{response_t2c_2['cypher_query']}")
        print(f"Answer: {response_t2c_2['answer']}")
        if response_t2c_2['cypher_query']:
            print("\n--- Cypher Explanation for Query 2 ---")
            explanation_q2 = explain_cypher_in_natural_language(response_t2c_2['cypher_query'], text2cypher_retriever.schema, openai_client)
            print(explanation_q2)
    else:
        print("\nSkipping Test Query 2 for Text2Cypher (MSFT or ESGU ETF data not available/processed, or etf_df lacks 'ticker' column).")

    # Test Query 3 (SPY & IT Companies Count)
    spy_exists = 'etf_df' in locals() and 'ticker' in etf_df.columns and not etf_df[etf_df['ticker'] == 'SPY'].empty
    if spy_exists:
        test_question_t2c_3 = "How many companies in the 'Information Technology' sector are held by the 'SPY' ETF in our sample data?"
        response_t2c_3 = text2cypher_retriever.query_kg(test_question_t2c_3)
        print("\n--- Test Query 3 (Text2Cypher) ---")
        print(f"Question: {response_t2c_3['question']}")
        print(f"Generated Cypher:\n{response_t2c_3['cypher_query']}")
        print(f"Answer: {response_t2c_3['answer']}")
        if response_t2c_3['cypher_query']:
            print("\n--- Cypher Explanation for Query 3 ---")
            explanation_q3 = explain_cypher_in_natural_language(response_t2c_3['cypher_query'], text2cypher_retriever.schema, openai_client)
            print(explanation_q3)
    else:
        print("\nSkipping Test Query 3 for Text2Cypher (SPY ETF data not available or etf_df lacks 'ticker' column).")

    # Test Query 4: Involving Investors
    nvda_processed = any(comp.get('ticker') == 'NVDA' and not comp.get('error') for comp in detailed_companies_data) if 'detailed_companies_data' in locals() else False
    ais_processed = any(comp.get('ticker') == 'AIS' and not comp.get('error') for comp in detailed_companies_data) if 'detailed_companies_data' in locals() else False
    investor_data_exists = 'investor_df' in locals() and not investor_df.empty
    if nvda_processed and ais_processed and investor_data_exists:
        test_question_t2c_4 = "Which investors have funded both NVIDIA (NVDA) and AI Startup Inc. (AIS)?"
        response_t2c_4 = text2cypher_retriever.query_kg(test_question_t2c_4)
        print("\n--- Test Query 4 (Text2Cypher - Common Investors) ---")
        print(f"Question: {response_t2c_4['question']}")
        print(f"Generated Cypher:\n{response_t2c_4['cypher_query']}")
        print(f"Answer: {response_t2c_4['answer']}")
        if response_t2c_4['cypher_query']:
            print("\n--- Cypher Explanation for Query 4 ---")
            explanation_q4 = explain_cypher_in_natural_language(response_t2c_4['cypher_query'], text2cypher_retriever.schema, openai_client)
            print(explanation_q4)
    else:
        print("\nSkipping Test Query 4 for Text2Cypher (NVDA, AIS, or investor data not available/processed).")

else:
    logging.warning("Neo4j driver or OpenAI client not initialized. Skipping Text2CypherRetriever tests.")




--- Test Query 1 (Text2Cypher) ---
Question: What is the latest news sentiment for Apple (AAPL) and its current sector?
Generated Cypher:
MATCH (c:Company {ticker: 'AAPL'})-[:OPERATES_IN_SECTOR]->(s:Sector),
      (c)-[:MENTIONED_IN_NEWS]->(n:NewsArticle)
RETURN n.inferredSentiment AS newsSentiment, s.name AS sector
ORDER BY n.publishTime DESC
LIMIT 1
Answer: The latest news sentiment for Apple (AAPL) is positive, and its current sector is Information Technology.

--- Cypher Explanation for Query 1 ---
1. The query starts by looking for a node labeled as `Company` with the property `ticker` equal to 'AAPL'.

2. It then traverses the relationship labeled as `OPERATES_IN_SECTOR` to find a related node labeled as `Sector`.

3. Additionally, the query traverses the relationship labeled as `MENTIONED_IN_NEWS` to find a related node labeled as `NewsArticle`.

4. The query applies a filter to only return the `inferredSentiment` property from the `NewsArticle` node and the `name` property fro

In [16]:
import logging
from typing import List, Dict, Any, Optional

class VectorSimilarityInvestmentRetriever:
    def __init__(self, vector_store: Any, openai_client: OpenAI):
        self.vector_store = vector_store
        self.client = openai_client
        self.answer_gen_model = "gpt-4.1-nano"

    def retrieve_similar_nodes(self, query_text: str, k: int = 5, score_threshold: Optional[float] = None) -> List[Dict[str, Any]]:
        if not self.vector_store:
            logging.error("Neo4jVector store instance not provided. Cannot retrieve.")
            return []

        logging.info(f"Performing vector similarity search for: '{query_text}' (k={k})")
        try:
            if score_threshold:
                 results_with_scores = self.vector_store.similarity_search_with_score(
                     query_text, k=k, score_threshold=score_threshold
                 )
            else:
                 results_with_scores = self.vector_store.similarity_search_with_score(query_text, k=k)

            processed_results = []
            for doc, score in results_with_scores:
                node_data = {"similarity_score": score, "text_content": doc.page_content}
                node_data.update(doc.metadata)
                processed_results.append(node_data)

            logging.info(f"Found {len(processed_results)} similar nodes via vector search.")
            return processed_results
        except Exception as e:
            logging.error(f"Error during vector similarity search: {e}")
            return []

    def synthesize_answer_from_vector_results(self, question: str, similar_nodes: List[Dict[str, Any]]) -> str:
        if not self.client:
            logging.warning("OpenAI client not initialized. Skipping answer synthesis.")
            return "Could not synthesize answer: OpenAI client not available."

        if not similar_nodes:
            logging.warning(f"No similar nodes found to synthesize answer for '{question}'.")
            return "I couldn't find any information directly matching your semantic query."

        context_items = []
        for i, node in enumerate(similar_nodes[:3]):
            item_desc = f"Entity {i+1} (Similarity: {node.get('similarity_score', 0.0):.2f}):\n"
            item_desc += f"  Type: {node.get('_labels', ['Unknown'])}\n"
            item_desc += f"  Name/Identifier: {node.get('name', node.get('ticker', 'N/A'))}\n"
            item_desc += f"  Key Text: {node.get('text_content', 'N/A')[:250]}...\n"
            if 'investmentProfileSummary' in node:
                 item_desc += f"  Investment Profile: {node['investmentProfileSummary'][:150]}...\n"
            if 'investmentFocus' in node:
                 item_desc += f"  Investment Focus: {node['investmentFocus'][:150]}...\n"
            context_items.append(item_desc)

        context_str = "\n".join(context_items)

        prompt_messages = [
            {"role": "system", "content": "You are a financial analyst assistant. Based on the user's question and the semantically similar entities retrieved from a knowledge graph, provide a concise and relevant natural language answer."},
            {"role": "user", "content": f"""
Original Question: {question}

Retrieved Semantically Similar Entities from Knowledge Graph:
{context_str}

Based on these entities, answer the original question.
Answer:
            """}
        ]

        try:
            completion = self.client.chat.completions.create(
                model=self.answer_gen_model,
                messages=prompt_messages,
                temperature=0.3
            )
            answer = completion.choices[0].message.content.strip()
            logging.info(f"Synthesized answer for '{question}' from vector results: {answer}")
            return answer
        except Exception as e:
            logging.error(f"LLM answer synthesis (vector) call failed: {e}")
            return "There was an issue generating the final answer."

    def query_kg_semantic(self, question: str, k: int = 3) -> Dict[str, Any]:
        logging.info(f"\n--- Vector Similarity Processing Question: \"{question}\" ---")
        similar_nodes = self.retrieve_similar_nodes(question, k=k)
        answer = self.synthesize_answer_from_vector_results(question, similar_nodes)

        return {
            "question": question,
            "retrieved_nodes_count": len(similar_nodes),
            "raw_retrieved_nodes": similar_nodes,
            "answer": answer,
            "retrieval_method": "VectorSimilarity"
        }

if 'vector_retriever_store' in locals() and vector_retriever_store and openai_client:
    vector_retriever = VectorSimilarityInvestmentRetriever(
        vector_store=vector_retriever_store,
        openai_client=openai_client
    )

    test_question_vec_1 = "Companies leading in AI-driven financial technology"
    response_vec_1 = vector_retriever.query_kg_semantic(test_question_vec_1, k=2)
    print("\n--- Test Query 1 (Vector Similarity) ---")
    print(f"Question: {response_vec_1['question']}")
    print(f"Answer: {response_vec_1['answer']}")

    test_question_vec_2 = "ETFs that offer exposure to renewable energy markets"
    response_vec_2 = vector_retriever.query_kg_semantic(test_question_vec_2, k=2)
    print("\n--- Test Query 2 (Vector Similarity) ---")
    print(f"Question: {response_vec_2['question']}")
    print(f"Answer: {response_vec_2['answer']}")
else:
    logging.warning("Neo4jVector store or OpenAI client not initialized. Skipping VectorSimilarityRetriever tests.")


--- Test Query 1 (Vector Similarity) ---
Question: Companies leading in AI-driven financial technology
Answer: Leading companies in AI-driven financial technology include JPMorgan Chase, which is investing heavily in AI and fintech modernization, and FactSet, which is enhancing its AI capabilities for portfolio analytics.

--- Test Query 2 (Vector Similarity) ---
Question: ETFs that offer exposure to renewable energy markets
Answer: The iShares ESG Aware MSCI USA ETF offers exposure to U.S. companies with strong environmental, social, and governance practices, aligning with renewable energy and sustainability themes. However, for dedicated renewable energy market exposure, specialized ETFs like the Invesco Solar ETF or the First Trust Global Wind Energy ETF may be more targeted options.


## Part 6: The Router Retriever - Strategically Selecting the Right Tool

We've now developed two powerful, yet distinct, retrieval strategies for our Investment Knowledge Graph:
1.  **Text2Cypher Retriever:** Excellent for precise, structured queries that translate well into specific graph traversals.
2.  **Vector Similarity Retriever:** Ideal for conceptual, semantic, or exploratory queries where the exact entities or relationships might not be known upfront.

In a sophisticated, production-grade AI system, how do you choose which retriever to use for a given user question? This is where a **Router Retriever** comes in.

**What is a Router Retriever?**

A Router Retriever is an intelligent component, often powered by an LLM itself or a set of heuristic rules, that analyzes an incoming natural language query and directs it to the most appropriate downstream retriever.

**Key Responsibilities of a Router:**
1.  **Query Analysis:** Understand the *intent* and *structure* of the user's question.
2.  **Strategy Selection:** Based on the analysis, choose the optimal retriever (Text2Cypher, Vector Similarity, Hybrid).
3.  **Reasoning (Optional but Recommended):** Log *why* a particular strategy was chosen.

**Building a Router (Conceptual Approach for this Notebook):**

Developing a full-fledged router is an advanced topic. For this notebook, we'll outline the conceptual approach:

1.  **LLM as a Router:** Use an LLM with a prompt that includes the user's query and descriptions of available retrieval strategies, asking it to choose the best one.
2.  **Heuristic/Rule-Based Routing:** Implement rules based on keywords or query structure.

**The Engineering Value for Your Systems:**
A well-designed router is critical for the performance, accuracy, and cost-effectiveness of a RAG system. It ensures the right tool is used for the job, leading to better results and more efficient resource utilization.

While we won't implement a full router class here, understanding its role is key to architecting robust Graph RAG solutions.

## Part 7: Practical Analysis: Graph RAG vs. Vector RAG for Investment Insights

Let's address a critical question: **When does the added complexity of a Knowledge Graph and Graph RAG provide a tangible advantage over a simpler Vector RAG system?**

**Scenario 1: Multi-Hop Relational Queries**

* **Question:** "Which ETFs hold both Apple (AAPL) and Microsoft (MSFT) and have an ESG focus?"
* **Graph RAG (Text2Cypher) Advantage:** High precision due to explicit relationship traversal. Can guarantee the ETF holds *both* stocks and meets ESG criteria from graph properties.
* **Traditional Vector RAG Challenge:** Lower precision, potential for false positives/negatives, difficulty in reliably verifying multi-condition relationships from text alone.
* **Key Takeaway for Engineering Leaders:** For queries demanding high accuracy on multiple, interconnected criteria, Graph RAG's ability to traverse verified relationships is superior.

**Scenario 2: Discovering Non-Obvious Connections & Network Effects**

* **Question:** "Are there common investors between NVIDIA (NVDA) and other companies in the 'Artificial intelligence' industry that have also received positive news sentiment recently?"
* **Graph RAG (Text2Cypher) Advantage:** Uncovers complex, indirect relationships and network effects by traversing multiple relationship types (investments, industry, news sentiment).
* **Traditional Vector RAG Challenge:** Very difficult to reliably identify and verify such multi-hop, multi-conditional network patterns from document similarity alone.
* **Key Takeaway for Product Managers:** Graph RAG can power features that provide unique "connected insights" – a strong differentiator for AI products.

**Scenario 3: Conceptual Search Combined with Structured Filtering**

* **Question:** "Find me innovative healthcare companies with strong recent performance (e.g., high forward P/E) and a focus on 'personalized medicine'."
* **Graph RAG (Hybrid: Vector Search + Text2Cypher) Advantage:** Combines exploratory power of semantic search (for "innovative personalized medicine") with precise structured graph filtering (for financial metrics).
* **Traditional Vector RAG Challenge:** May struggle to accurately apply quantitative filters directly during vector search.
* **Key Takeaway for CTOs:** A hybrid Graph RAG architecture offers versatility, allowing the system to choose the best path – semantic, structural, or both.

**Conclusion of Comparison:**

The investment in building a Knowledge Graph pays off when your application needs to:
* Answer questions dependent on **complex relationships and multiple hops**.
* Ensure **high precision and verifiability** of these relationships.
* Combine **semantic understanding with structured filtering** seamlessly.
* Uncover **network effects and non-obvious connections**.

For many advanced business intelligence and investment analysis use cases, the depth and precision offered by Graph RAG can provide a significant competitive advantage.

## Conclusion & Next Steps: Building ROI-Driven AI with Graph RAG

We've journeyed from foundational company and investment data to a sophisticated Investment Knowledge Graph, enriched by LLMs and queryable through advanced RAG techniques.

**Key Takeaways for Your AI Strategy:**

1.  **Knowledge Graphs Unlock Deeper Insights:** Structuring data around entities and relationships allows for complex, multi-faceted questions, crucial in domains like finance.
2.  **LLMs as Graph Augmenters & Interpreters:** LLMs enrich KGs (sentiment, summaries) and enable natural language access (Text2Cypher).
3.  **Graph RAG for Precision and Context:** For queries dependent on verified relationships, Graph RAG offers superior precision.
4.  **Vector Search on Graphs for Exploration:** Integrating vector indexes provides powerful semantic search for conceptual queries.
5.  **Strategic Query Routing is Key:** A router selecting the best retrieval strategy is crucial for performance, accuracy, and cost.
6.  **ROI in Advanced RAG:** Translates into faster insights, discovery of non-obvious patterns, powerful AI product features, and better decision support.

**From Proof-of-Concept to Production:**

As an AI engineering consultant, I help organizations navigate from AI experiments to robust, scalable, ROI-driven production systems. The concepts here – KG construction, LLM enrichment, advanced RAG, and query routing – are foundational for high-impact AI solutions.

**Potential Next Steps for Your Organization:**

* **Identify High-Value Use Cases:** Where do complex relationships and contextual insights offer the biggest leverage?
* **Data Audit & Modeling:** Assess your data sources for KG potential.
* **Pilot Project:** Start with a focused pilot to demonstrate Graph RAG value.
* **Performance & Scalability Planning:** Consider optimized ingestion, query tuning, and MLOps for LLM pipelines.
* **Measure Business Impact:** Define KPIs to track outcomes from improved insights.

The path to advanced AI involves innovative techniques and sound engineering. Knowledge Graphs, thoughtfully applied with modern LLMs, are a significant step in transforming data into strategic intelligence.

---

Thank you for working through this notebook. I hope it has provided valuable insights into the power of Advanced Graph RAG. If you're looking to explore how these concepts can be tailored and implemented to solve your specific business challenges, I'm here to help.

*Saumil Srivastava*
*AI Engineering Consultant*
*[Book a consultation](https://www.saumilsrivastava.ai/book-consultation)*